# Assignment 2 - Data Mining

## 2.

Needed libraries

In [2]:
# ! pip install transformers
# ! pip install datasets
# ! pip install seqeval
# ! pip install evaluate
# ! pip install torch
# ! pip install accelerate
# ! pip install transformers[torch]



In [3]:
from datasets import Dataset
from transformers import AutoTokenizer
import torch
import accelerate
import evaluate


# Importing training set

In [4]:
# Define the data format
data = {
    "tokens": [],  # List of tokens
    "labels": [],  # List of integer labels
}

label_to_int = {}  # Dictionary to map original labels to integers
label_names = []  # List to map integers to original labels

train_path = "W-NUT_data/wnut17train.conll"


# Read the .conll file and populate the data dictionary
with open(train_path, "r") as file:
    lines = file.readlines()
    tokens, labels = [], []
    for line in lines:
        parts = line.strip().split()
        if parts:
            token, label = parts
            tokens.append(token)

            # Check if the label is already in the mapping dictionary
            if label not in label_to_int:
                label_to_int[label] = len(label_to_int)
                label_names.append(label)

            labels.append(label_to_int[label])
        else:
            data["tokens"].append(tokens)
            data["labels"].append(labels)
            tokens, labels = [], []  # Reset for the next sentence

# Create a custom dataset
train = Dataset.from_dict(data)

In [5]:
print(train)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 3394
})


# Importing validation set

In [6]:
# Define the data format for the validation set
validation_data = {
    "tokens": [],  # List of tokens
    "labels": [],  # List of integer labels
}

validation_path = "W-NUT_data/emerging.dev.conll"  # Replace with the actual path to your validation set .conll file

# Read the .conll file for the validation set and populate the data dictionary
with open(validation_path, "r") as file:
    lines = file.readlines()
    tokens, labels = [], []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:  # Check if there are both token and label
            token, label = parts
            tokens.append(token)

            if label not in label_to_int:
                label_to_int[label] = len(label_names)
                label_names.append(label)

            labels.append(label_to_int[label])
        else:
            if parts:  # Handle lines with only one value
                # Decide how to handle lines with one value (e.g., set a default label)
                token = parts[0]
                label = "O"  # You can replace this with an appropriate default label
                tokens.append(token)

                if label not in label_to_int:
                    label_to_int[label] = len(label_names)
                    label_names.append(label)

                labels.append(label_to_int[label])
            else:
                validation_data["tokens"].append(tokens)
                validation_data["labels"].append(labels)
                tokens, labels = [], []  # Reset for the next sentence

# If there's data left to process
if tokens:
    validation_data["tokens"].append(tokens)
    validation_data["labels"].append(labels)

# Create a custom dataset for the validation set
validation = Dataset.from_dict(validation_data)


In [7]:
print(validation)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 1009
})


Importing test set

In [8]:
# Define the data format for the test set
test_data = {
   "tokens": [],  # List of tokens
   "labels": [],  # List of integer labels
}

test_path = "W-NUT_data/emerging.test.annotated"  # Replace with the actual path to your test set .conll file

# Read the .conll file for the test set and populate the data dictionary
with open(test_path, "r") as file:
   lines = file.readlines()
   tokens, labels = [], []
   for line in lines:
       parts = line.strip().split()
       if len(parts) == 2:  # Check if there are both token and label
           token, label = parts
           tokens.append(token)

           if label not in label_to_int:
               label_to_int[label] = len(label_names)
               label_names.append(label)

           labels.append(label_to_int[label])
       else:
           if parts:  # Handle lines with only one value
               # Decide how to handle lines with one value (e.g., set a default label)
               token = parts[0]
               label = "O"  # You can replace this with an appropriate default label
               tokens.append(token)

               if label not in label_to_int:
                   label_to_int[label] = len(label_names)
                   label_names.append(label)

               labels.append(label_to_int[label])
           else:
               test_data["tokens"].append(tokens)
               test_data["labels"].append(labels)
               tokens, labels = [], []  # Reset for the next sentence

# If there's data left to process
if tokens:
   test_data["tokens"].append(tokens)
   test_data["labels"].append(labels)

# Create a custom dataset for the test set
test = Dataset.from_dict(test_data)


In [9]:
print(test)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 1287
})


In [10]:
print(label_to_int)

{'O': 0, 'B-location': 1, 'I-location': 2, 'B-group': 3, 'B-corporation': 4, 'B-person': 5, 'B-creative-work': 6, 'B-product': 7, 'I-person': 8, 'I-creative-work': 9, 'I-corporation': 10, 'I-group': 11, 'I-product': 12}


In [11]:
print(label_names)

['O', 'B-location', 'I-location', 'B-group', 'B-corporation', 'B-person', 'B-creative-work', 'B-product', 'I-person', 'I-creative-work', 'I-corporation', 'I-group', 'I-product']


# Realizing a dataset dictionary

In [12]:
from datasets import DatasetDict

raw_datasets = DatasetDict({
    'train': train,
    'validation': validation,
    'test': test
})

In [13]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 1287
    })
})


# Verify the initial alignment between tokens and labels.

In [14]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["labels"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

@paulwalk It 's the view from where I 'm living for two weeks . Empire     State      Building   = ESB        . Pretty bad storm here last evening . 
O         O  O  O   O    O    O     O O  O      O   O   O     O B-location I-location I-location O B-location O O      O   O     O    O    O       O 


# Importing the needed tokenizer

In [15]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [16]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '@',
 'p',
 '##aul',
 '##walk',
 'It',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'I',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'Empire',
 'State',
 'Building',
 '=',
 'E',
 '##SB',
 '.',
 'Pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

# Defining the fucìnction to align labels and tokens properly

In [17]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [18]:
labels = raw_datasets["train"][0]["labels"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]


Defining the function to apply the previous function on all the dataset

In [19]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [20]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [21]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1287
    })
})


Data collation

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [23]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    1,    2,    2,    0,
            1,    2,    0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    3,    4,    4,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0, -100, -100]])

In [24]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


# 3 Evaluation and metrics

In [25]:
metric = evaluate.load("seqeval")

In [26]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# Task 4

In [27]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [28]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
model.config.num_labels

13

In [30]:
from huggingface_hub import notebook_login

notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
from huggingface_hub import notebook_login

notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [34]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.223547,0.601732,0.172886,0.268599,0.928945
2,0.120600,0.236243,0.583123,0.191957,0.288833,0.930905
3,0.051000,0.235401,0.550000,0.223466,0.317807,0.933015


TrainOutput(global_step=1275, training_loss=0.07559682210286459, metrics={'train_runtime': 463.3458, 'train_samples_per_second': 21.975, 'train_steps_per_second': 2.752, 'total_flos': 289506808910040.0, 'train_loss': 0.07559682210286459, 'epoch': 3.0})